In [ ]:
# Konlpy 설치
!pip install konlpy


In [ ]:
import pandas as pd
from collections import defaultdict
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

url = 'https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/melon_lyric_preprocess.csv'
df = pd.read_csv(url)


In [ ]:
okt = Okt()

co_occ_df = pd.DataFrame(columns=['year', 'genre', 'word1', 'word2', 'count'])

years = df['year'].unique().tolist()
for year in range(2010, 2021):
    year_df = df.loc[df['year'] == year]
    year_co_occ_df = pd.DataFrame(columns=['year', 'genre', 'word1', 'word2', 'count'])

    for index, row in year_df.iterrows():
        co_occurenced_pair = defaultdict(int)
        genre = row['genre']
        lyric = row['lyric']
        words = [word[0] for word in okt.pos(lyric) if word[1] == 'Noun' or word[1] == 'Adjective']

        for i in range(len(words)):
            for j in range(i+1, len(words)):
                if words[i] == words[j]:
                    continue
                tmp = tuple(sorted([words[i], words[j]]))
                co_occurenced_pair[tmp] += 1

        current = pd.DataFrame(
            {
                "year": year,
                'genre': genre,
                'word1': [pair[0] for pair in co_occurenced_pair],
                'word2': [pair[1] for pair in co_occurenced_pair],
                'count': [co_occurenced_pair[pair] for pair in co_occurenced_pair],
            }
        ) 

        year_co_occ_df = pd.concat([year_co_occ_df, current])

    year_co_occ_df['count'] = year_co_occ_df.groupby(['year', 'genre', 'word1', 'word2'])['count'].transform('sum')
    year_co_occ_df = year_co_occ_df.drop_duplicates(subset=['year', 'genre', 'word1', 'word2'])
    year_co_occ_df = year_co_occ_df.loc[year_co_occ_df['count'] >= 10]

    co_occ_df = pd.concat([co_occ_df, year_co_occ_df])

In [ ]:
co_occ_df.to_csv('lyric_co_occurenced_2010-2020.csv')
from google.colab import files
files.download('lyric_co_occurenced_2010-2020.csv')